In [1]:
import json
import pandas as pd
import requests
import utils

In [2]:
api_key = utils.api_key()
url = 'https://maps.googleapis.com/maps/api/place/textsearch/json?'

def dataframe_from_area_code(query):
    r = requests.get(url + 'query=' + query + '&key=' + api_key)
    x = r.json()
    y = x['results']
    df = pd.DataFrame(y)
    return df

global final_dataset

In [3]:
# west LA zip codes
zip_codes_west = ['90266', '90267', '90291', '90292', '90293','90401','90402','90403','90404','90405']
places = ['smoke shops', 'liquor stores', 'marijuana dispenseries', 'schools']

In [4]:
final_dataset = pd.DataFrame()

for i in range(len(places)):
    for j in range(len(zip_codes_west)):
        a = pd.DataFrame(dataframe_from_area_code(places[i] + ' near ' + zip_codes_west[j]))
        a['shops'] = places[i]
        a['zip'] = zip_codes_west[j]
        final_dataset = pd.concat([final_dataset, a], axis=0, sort=True)

In [5]:
# clean up and create final dataframe
final_dataset.drop(['id', 'icon', 'place_id', 'opening_hours', 
                    'plus_code', 'reference', 'photos', 'price_level', 
                    'rating', 'types', 'user_ratings_total'], axis=1, inplace=True)
final_dataset.rename(columns = {'formatted_address': 'address', 'geometry': 'lat_long'}, inplace=True)
final_dataset.drop_duplicates(subset='address', keep='first', inplace=True)
final_dataset.reset_index(drop=True, inplace=True)
print(final_dataset.shape)
final_dataset.head()

(230, 5)


,address,lat_long,name,shops,zip
0,"2208 Artesia Blvd A, Redondo Beach, CA 90278, ...","{'location': {'lat': 33.8726545, 'lng': -118.3...",The Yellow Smoke Shop,smoke shops,90266
1,"1005 N Aviation Blvd, Redondo Beach, CA 90278,...","{'location': {'lat': 33.8857648, 'lng': -118.3...",Manhattan Beach Smoke Shop,smoke shops,90266
2,"1710 Aviation Blvd, Redondo Beach, CA 90278, U...","{'location': {'lat': 33.8710098, 'lng': -118.3...",Redondo Cigar & Tobacco,smoke shops,90266
3,"2739 Manhattan Beach Blvd, Redondo Beach, CA 9...","{'location': {'lat': 33.888337, 'lng': -118.36...",SMOKE ZONE,smoke shops,90266
4,"432 CA-1, Hermosa Beach, CA 90254, United States","{'location': {'lat': 33.8581076, 'lng': -118.3...",Cigar Hookah & Beyond Smoke and Vape Shop,smoke shops,90266


In [6]:
count1 = pd.DataFrame(columns=places, index=zip_codes_west)
count1[count1 != 0] = 0
for i in range(len(final_dataset['zip'])):
    for j in range(len(zip_codes_west)):
        if final_dataset['zip'][i] == zip_codes_west[j]:
            b = zip_codes_west[j]
            for z in range(len(places)):
                if final_dataset['shops'][i] == places[z]:
                    a = count1.loc[b][places[z]]
                    count1.loc[b][places[z]] = a+1
count1                                      

,smoke shops,liquor stores,marijuana dispenseries,schools
90266,20,18,3,19
90267,5,7,0,7
90291,20,20,0,17
90292,6,11,0,16
90293,6,9,2,13
90401,3,12,2,1
90402,1,4,0,0
90403,0,0,0,0
90404,3,1,0,0
90405,0,3,1,0


# Western LA Counts

In [7]:
print(list(count1['smoke shops']))
print(list(count1['liquor stores']))
print(list(count1['marijuana dispenseries']))
print(list(count1['schools']))

[20, 5, 20, 6, 6, 3, 1, 0, 3, 0]
[18, 7, 20, 11, 9, 12, 4, 0, 1, 3]
[3, 0, 0, 0, 2, 2, 0, 0, 0, 1]
[19, 7, 17, 16, 13, 1, 0, 0, 0, 0]
